In [0]:
spark.conf.set(
    "fs.azure.account.key.<storage-account>.dfs.core.windows.net",
    dbutils.secrets.get(scope="<scope>", key="<storage-account-access-key>"))

In [0]:
dbutils.help()

This module provides various utilities for users to interact with the rest of Databricks.
 credentials: DatabricksCredentialUtils -> Utilities for interacting with credentials within notebooks data: DataUtils -> Utilities for understanding and interacting with datasets (EXPERIMENTAL) fs: DbfsUtils -> Manipulates the Databricks filesystem (DBFS) from the console jobs: JobsUtils -> Utilities for leveraging jobs features library: LibraryUtils -> Utilities for session isolated libraries meta: MetaUtils -> Methods to hook into the compiler (EXPERIMENTAL) notebook: NotebookUtils -> Utilities for the control flow of a notebook (EXPERIMENTAL) preview: Preview -> Utilities under preview category secrets: SecretUtils -> Provides utilities for leveraging secrets within notebooks widgets: WidgetsUtils -> Methods to create and get bound value of input widgets inside notebooks

In [0]:
dbutils.secrets.help()

Provides utilities for leveraging secrets within notebooks.
Databricks documentation for more info.
 get(scope: String, key: String): String -> Gets the string representation of a secret value with scope and key getBytes(scope: String, key: String): byte[] -> Gets the bytes representation of a secret value with scope and key list(scope: String): Seq -> Lists secret metadata for secrets within a scope listScopes: Seq -> Lists secret scopes

In [0]:
scopes = dbutils.secrets.listScopes()
display(scopes)

name
optumscope


In [0]:
keys = dbutils.secrets.list("optumscope")
display(keys)

key
optum-adlskey
optum-blobkey
optum-blobstring


In [0]:
# spark.conf.set(
#     "fs.azure.account.key.<storage-account>.dfs.core.windows.net",
#     dbutils.secrets.get(scope="<scope>", key="<storage-account-access-key>"))

In [0]:
spark.conf.set(
    "fs.azure.account.key.optumadlsstrg.dfs.core.windows.net",
    dbutils.secrets.get(scope="optumscope", key="optum-adlskey"))

In [0]:
dbutils.fs.ls("abfss://optum@optumadlsstrg.dfs.core.windows.net")

[FileInfo(path='abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/', name='bronze/', size=0, modificationTime=1742050324000),
 FileInfo(path='abfss://optum@optumadlsstrg.dfs.core.windows.net/gold/', name='gold/', size=0, modificationTime=1742050341000),
 FileInfo(path='abfss://optum@optumadlsstrg.dfs.core.windows.net/silver/', name='silver/', size=0, modificationTime=1742050334000)]

In [0]:
display(dbutils.fs.ls("abfss://optum@optumadlsstrg.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/,bronze/,0,1742050324000
abfss://optum@optumadlsstrg.dfs.core.windows.net/gold/,gold/,0,1742050341000
abfss://optum@optumadlsstrg.dfs.core.windows.net/silver/,silver/,0,1742050334000


In [0]:
display(dbutils.fs.ls("abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze"))

path,name,size,modificationTime
abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/Patient_records.csv,Patient_records.csv,5110,1742106535000
abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/claims.json,claims.json,16385,1742106535000
abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/disease.csv,disease.csv,1489,1742106535000
abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/group.csv,group.csv,4390,1742106535000
abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/hospital.csv,hospital.csv,1328,1742106535000
abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/subgroup.csv,subgroup.csv,561,1742106535000
abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/subscriber.csv,subscriber.csv,12061,1742106535000


# Connector code

In [0]:
def adlsconnector():
    spark.conf.set("fs.azure.account.key.optumadlsstrg.dfs.core.windows.net",
    dbutils.secrets.get(scope="optumscope", key="optum-adlskey"))
    print("adls connection successful")

In [0]:
def readbronzedata(fn):
    adlsconnector()
    df = spark.read.format("csv").option("header", "true").load("abfss://optum@optumadlsstrg.dfs.core.windows.net/bronze/"+fn)
    return(df)

In [0]:
df = readbronzedata("hospital.csv")
df.show(5,False)

adls connection successful
+-----------+-----------------------------------------------------------------+----------+----------+-------+
|Hospital_id|Hospital_name                                                    |city      |state     |country|
+-----------+-----------------------------------------------------------------+----------+----------+-------+
|H1000      |All India Institute of Medical Sciences                          |New Delhi |NaN       |India  |
|H1001      |Medanta The Medicity                                             |Gurgaon   |Haryana   |India  |
|H1002      |The Christian Medical College                                    |Vellore   |Tamil Nadu|India  |
|H1003      |PGIMER - Postgraduate Institute of Medical Education and Research|Chandigarh|Haryana   |India  |
|H1004      |Apollo Hospital - Chennai                                        |Chennai   |Tamil Nadu|India  |
+-----------+-----------------------------------------------------------------+----------+---

## Hospital Transformations

In [0]:
%run "/Workspace/Shared/Optum_Project/connectors"

In [0]:
%run "/Workspace/Shared/Optum_Project/dataprecheck"

In [0]:
df = readbronzedata("hospital.csv")
df.display()

adls connection successful


Hospital_id,Hospital_name,city,state,country
H1000,All India Institute of Medical Sciences,New Delhi,NaN,India
H1001,Medanta The Medicity,Gurgaon,Haryana,India
H1002,The Christian Medical College,Vellore,Tamil Nadu,India
H1003,PGIMER - Postgraduate Institute of Medical Education and Research,Chandigarh,Haryana,India
H1004,Apollo Hospital - Chennai,Chennai,Tamil Nadu,India
H1005,P. D. Hinduja National Hospital & Medical Research Centre,Mumbai,Maharashtra,India
H1006,Breach Candy Hospital,Mumbai,Maharashtra,India
H1007,Fortis Flt. Lt. Rajan Dhall Hospital,New Delhi,NaN,India
H1008,King Edward Memorial Hospital,Mumbai,Maharashtra,India
H1009,Indraprastha Apollo Hospital,Delhi,NaN,India


In [0]:
missingvaluecheck(df)

+-----------+-------------+----+-----+-------+
|Hospital_id|Hospital_name|city|state|country|
+-----------+-------------+----+-----+-------+
|          0|            0|   0|    4|      0|
+-----------+-------------+----+-----+-------+



In [0]:
df = df.replace("NaN", None)       # first replace the NaN with None i.e. null so that we can handle null values
df.show(5)

+-----------+--------------------+----------+----------+-------+
|Hospital_id|       Hospital_name|      city|     state|country|
+-----------+--------------------+----------+----------+-------+
|      H1000|All India Institu...| New Delhi|      NULL|  India|
|      H1001|Medanta The Medicity|   Gurgaon|   Haryana|  India|
|      H1002|The Christian Med...|   Vellore|Tamil Nadu|  India|
|      H1003|PGIMER - Postgrad...|Chandigarh|   Haryana|  India|
|      H1004|Apollo Hospital -...|   Chennai|Tamil Nadu|  India|
+-----------+--------------------+----------+----------+-------+
only showing top 5 rows


In [0]:
df = df.fillna({'state':'UT'})
df.show(5,False)

+-----------+-----------------------------------------------------------------+----------+----------+-------+
|Hospital_id|Hospital_name                                                    |city      |state     |country|
+-----------+-----------------------------------------------------------------+----------+----------+-------+
|H1000      |All India Institute of Medical Sciences                          |New Delhi |UT        |India  |
|H1001      |Medanta The Medicity                                             |Gurgaon   |Haryana   |India  |
|H1002      |The Christian Medical College                                    |Vellore   |Tamil Nadu|India  |
|H1003      |PGIMER - Postgraduate Institute of Medical Education and Research|Chandigarh|Haryana   |India  |
|H1004      |Apollo Hospital - Chennai                                        |Chennai   |Tamil Nadu|India  |
+-----------+-----------------------------------------------------------------+----------+----------+-------+
only showi

In [0]:
df = df.replace("New Delhi", "Delhi")
df.show(5)

+-----------+--------------------+----------+----------+-------+
|Hospital_id|       Hospital_name|      city|     state|country|
+-----------+--------------------+----------+----------+-------+
|      H1000|All India Institu...|     Delhi|        UT|  India|
|      H1001|Medanta The Medicity|   Gurgaon|   Haryana|  India|
|      H1002|The Christian Med...|   Vellore|Tamil Nadu|  India|
|      H1003|PGIMER - Postgrad...|Chandigarh|   Haryana|  India|
|      H1004|Apollo Hospital -...|   Chennai|Tamil Nadu|  India|
+-----------+--------------------+----------+----------+-------+
only showing top 5 rows


## Group Transformation

In [0]:
grp_df = readbronzedata("group.csv")
grp_df.display()

adls connection successful


country,premium_written,zip_code,grp_id,grp_name,grp_type,city
India,72000,482018,GRP101,Life Insurance Corporation of India,Govt.,Mumbai
India,45000,482049,GRP102,HDFC Standard Life Insurance Co. Ltd.,Private,Mumbai
India,64000,482030,GRP103,Max Life Insurance Co. Ltd.,Private,Delhi
India,59000,482028,GRP104,ICICI Prudential Life Insurance Co. Ltd.,Private,Mumbai
India,37000,482014,GRP105,Kotak Mahindra Life Insurance Co. Ltd.,Private,Mumbai
India,89000,482011,GRP106,Aditya Birla Sun Life Insurance Co. Ltd.,Private,Mumbai
India,70000,482006,GRP107,TATA AIG Life Insurance Co. Ltd.,Private,Mumbai
India,52000,482034,GRP108,SBI Life Insurance Co. Ltd.,Private,Mumbai
India,78000,482032,GRP109,Exide Life Insurance Co. Ltd.,Private,Bangalore
India,48000,482015,GRP110,Bajaj Allianz Life Insurance Co. Ltd.,Private,Pune


In [0]:
missingvaluecheck(grp_df)

+-------+---------------+--------+------+--------+--------+----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|
+-------+---------------+--------+------+--------+--------+----+
|      0|              0|       0|     0|       0|       0|   0|
+-------+---------------+--------+------+--------+--------+----+



In [0]:
tocheckduplicates(grp_df)

+-------+---------------+--------+------+--------+--------+----+-----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|count|
+-------+---------------+--------+------+--------+--------+----+-----+
+-------+---------------+--------+------+--------+--------+----+-----+



## Subgroup transformation

In [0]:
subgrp_df = readbronzedata("subgroup.csv")

adls connection successful


In [0]:
subgrp_df.display()

subgrp_sk,subgrp_name,monthly_premium,subgrp_id
S101,Deficiency Diseases,3000,"GRP101,GRP105"
S102,Accident,1000,"GRP110,GRP150,GRP136"
S103,Physiology,2000,"GRP122,GRP108,GRP138,GRP148"
S104,Therapy,1500,"GRP103,GRP113,GRP123,GRP133,GRP143"
S105,Allergies,2300,"GRP153,GRP104,GRP114,GRP124"
S106,Self inflicted,1200,"GRP117,GRP127,GRP137,GRP147,GRP157"
S107,Cancer,3200,"GRP151,GRP131,GRP141,GRP121"
S108,Infectious disease,1500,"GRP130,GRP104,GRP109"
S109,Hereditary,2000,"GRP102,GRP112,GRP132,GRP142,GRP152"
S110,Viral,1000,"GRP143,GRP147,GRP126"


In [0]:
subgrp_df = subgrp_df.withColumn("subgrp_id",split(col("subgrp_id"),","))

In [0]:
subgrp_df.display()

subgrp_sk,subgrp_name,monthly_premium,subgrp_id
S101,Deficiency Diseases,3000,GRP101
S101,Deficiency Diseases,3000,GRP105
S102,Accident,1000,GRP110
S102,Accident,1000,GRP150
S102,Accident,1000,GRP136
S103,Physiology,2000,GRP122
S103,Physiology,2000,GRP108
S103,Physiology,2000,GRP138
S103,Physiology,2000,GRP148
S104,Therapy,1500,GRP103


In [0]:
# Explode --> Transform each element of the list into new row

subgrp_df = subgrp_df.withColumn('subgrp_id', explode(col('subgrp_id')))

In [0]:
subgrp_df.display()

subgrp_sk,subgrp_name,monthly_premium,subgrp_id
S101,Deficiency Diseases,3000,GRP101
S101,Deficiency Diseases,3000,GRP105
S102,Accident,1000,GRP110
S102,Accident,1000,GRP150
S102,Accident,1000,GRP136
S103,Physiology,2000,GRP122
S103,Physiology,2000,GRP108
S103,Physiology,2000,GRP138
S103,Physiology,2000,GRP148
S104,Therapy,1500,GRP103


## Disease Transformation

In [0]:
ds_df = readbronzedata("disease.csv")

adls connection successful


In [0]:
ds_df.display()

subgrp_id,disease_id,disease_name
S101,110001,Beriberi
S101,110002,Scurvy
S101,110003,Goitre
S101,110004,Osteoporosis
S101,110005,Rickets
S101,110006,Anaemia
S102,110007,Fractures
S102,110008,Heart Attack
S102,110009,Burns
S102,110010,Choking


In [0]:
tocheckduplicates(ds_df)

+---------+----------+------------+-----+
|subgrp_id|disease_id|disease_name|count|
+---------+----------+------------+-----+
+---------+----------+------------+-----+



In [0]:
missingvaluecheck(ds_df)

+---------+----------+------------+
|subgrp_id|disease_id|disease_name|
+---------+----------+------------+
|        0|         0|           0|
+---------+----------+------------+



## Patient Records Transformation

In [0]:
pr_df = readbronzedata("Patient_records.csv")

adls connection successful


In [0]:
pr_df.display()

Patient_id,Patient_name,patient_gender,patient_birth_date,patient_phone,disease_name,city,hospital_id
187158,Harbir,Female,1924-06-30,+91 0112009318,Galactosemia,Rourkela,H1001
112766,Brahmdev,Female,1948-12-20,+91 1727749552,Bladder cancer,Tiruvottiyur,H1016
199252,Ujjawal,Male,1980-04-16,+91 8547451606,Kidney cancer,Berhampur,H1009
133424,Ballari,Female,1969-09-25,+91 0106026841,Suicide,Bihar Sharif,H1017
172579,Devnath,Female,1946-05-01,+91 1868774631,Food allergy,Bidhannagar,H1019
171320,Atasi,Male,1967-10-02,+91 9747336855,Whiplash,Amravati,H1013
107794,Manish,Male,1967-06-06,+91 4354294043,Sunbathing,Panvel,H1004
130339,Aakar,Female,1925-03-05,+91 2777633911,Drug consumption,Bihar Sharif,H1000
110377,Gurudas,Male,1945-05-06,+91 1232859381,Dengue,Kamarhati,H1001
149367,null,Male,1925-06-12,+91 1780763280,Head banging,Bangalore,H1013


In [0]:
list(pr_df.columns)

['Patient_id',
 'Patient_name',
 'patient_gender',
 'patient_birth_date',
 'patient_phone',
 'disease_name',
 'city',
 'hospital_id']

In [0]:
sel_col = ['Patient_id',
 'Patient_name',
 'patient_gender',
 'patient_phone',
 'disease_name',
 'city',
 'hospital_id']

sel_pr_df = pr_df.select(sel_col)
display(sel_pr_df)

Patient_id,Patient_name,patient_gender,patient_phone,disease_name,city,hospital_id
187158,Harbir,Female,+91 0112009318,Galactosemia,Rourkela,H1001
112766,Brahmdev,Female,+91 1727749552,Bladder cancer,Tiruvottiyur,H1016
199252,Ujjawal,Male,+91 8547451606,Kidney cancer,Berhampur,H1009
133424,Ballari,Female,+91 0106026841,Suicide,Bihar Sharif,H1017
172579,Devnath,Female,+91 1868774631,Food allergy,Bidhannagar,H1019
171320,Atasi,Male,+91 9747336855,Whiplash,Amravati,H1013
107794,Manish,Male,+91 4354294043,Sunbathing,Panvel,H1004
130339,Aakar,Female,+91 2777633911,Drug consumption,Bihar Sharif,H1000
110377,Gurudas,Male,+91 1232859381,Dengue,Kamarhati,H1001
149367,null,Male,+91 1780763280,Head banging,Bangalore,H1013


In [0]:
missingvaluecheck(sel_pr_df)

+----------+------------+--------------+-------------+------------+----+-----------+
|Patient_id|Patient_name|patient_gender|patient_phone|disease_name|city|hospital_id|
+----------+------------+--------------+-------------+------------+----+-----------+
|         0|          17|             0|            2|           0|   0|          0|
+----------+------------+--------------+-------------+------------+----+-----------+



In [0]:
pr_df.filter(pr_df['patient_birth_date'].isNull()).count()

0

In [0]:
pr_df = pr_df.fillna({"Patient_name": "Visitor/NA"})
pr_df.display()

Patient_id,Patient_name,patient_gender,patient_birth_date,patient_phone,disease_name,city,hospital_id
187158,Harbir,Female,1924-06-30,+91 0112009318,Galactosemia,Rourkela,H1001
112766,Brahmdev,Female,1948-12-20,+91 1727749552,Bladder cancer,Tiruvottiyur,H1016
199252,Ujjawal,Male,1980-04-16,+91 8547451606,Kidney cancer,Berhampur,H1009
133424,Ballari,Female,1969-09-25,+91 0106026841,Suicide,Bihar Sharif,H1017
172579,Devnath,Female,1946-05-01,+91 1868774631,Food allergy,Bidhannagar,H1019
171320,Atasi,Male,1967-10-02,+91 9747336855,Whiplash,Amravati,H1013
107794,Manish,Male,1967-06-06,+91 4354294043,Sunbathing,Panvel,H1004
130339,Aakar,Female,1925-03-05,+91 2777633911,Drug consumption,Bihar Sharif,H1000
110377,Gurudas,Male,1945-05-06,+91 1232859381,Dengue,Kamarhati,H1001
149367,Visitor/NA,Male,1925-06-12,+91 1780763280,Head banging,Bangalore,H1013


In [0]:
pr_df= pr_df.drop("patient_phone")

In [0]:
pr_df = pr_df.withColumn('patient_age', round(months_between(current_date(), pr_df.patient_birth_date)/12,0).cast('integer'))

In [0]:
pr_df.display()

Patient_id,Patient_name,patient_gender,patient_birth_date,disease_name,city,hospital_id,patient_age
187158,Harbir,Female,1924-06-30,Galactosemia,Rourkela,H1001,101
112766,Brahmdev,Female,1948-12-20,Bladder cancer,Tiruvottiyur,H1016,76
199252,Ujjawal,Male,1980-04-16,Kidney cancer,Berhampur,H1009,45
133424,Ballari,Female,1969-09-25,Suicide,Bihar Sharif,H1017,55
172579,Devnath,Female,1946-05-01,Food allergy,Bidhannagar,H1019,79
171320,Atasi,Male,1967-10-02,Whiplash,Amravati,H1013,57
107794,Manish,Male,1967-06-06,Sunbathing,Panvel,H1004,58
130339,Aakar,Female,1925-03-05,Drug consumption,Bihar Sharif,H1000,100
110377,Gurudas,Male,1945-05-06,Dengue,Kamarhati,H1001,80
149367,Visitor/NA,Male,1925-06-12,Head banging,Bangalore,H1013,100


In [0]:
pr_df = pr_df.drop("patient_birth_date")

In [0]:
pr_df.display() 

Patient_id,Patient_name,patient_gender,disease_name,city,hospital_id,patient_age
187158,Harbir,Female,Galactosemia,Rourkela,H1001,101
112766,Brahmdev,Female,Bladder cancer,Tiruvottiyur,H1016,76
199252,Ujjawal,Male,Kidney cancer,Berhampur,H1009,45
133424,Ballari,Female,Suicide,Bihar Sharif,H1017,55
172579,Devnath,Female,Food allergy,Bidhannagar,H1019,79
171320,Atasi,Male,Whiplash,Amravati,H1013,57
107794,Manish,Male,Sunbathing,Panvel,H1004,58
130339,Aakar,Female,Drug consumption,Bihar Sharif,H1000,100
110377,Gurudas,Male,Dengue,Kamarhati,H1001,80
149367,Visitor/NA,Male,Head banging,Bangalore,H1013,100


## Claims Transformation

In [0]:
df = readbronzedata_json("claims.json")

adls connection successful


In [0]:
df.display()

Claim_Or_Rejected,SUB_ID,claim_amount,claim_date,claim_id,claim_type,disease_name,patient_id
N,SUBID1000,79874,1949-03-14,0,claims of value,Galactosemia,187158
N,SUBID10001,151142,1970-03-16,1,claims of policy,Bladder cancer,112766
N,SUBID10002,59924,2008-02-03,2,claims of value,Kidney cancer,199252
N,SUBID10003,143120,1995-02-08,3,claims of fact,Suicide,133424
Y,SUBID10004,168634,1967-05-23,4,claims of value,Food allergy,172579
N,SUBID10005,64840,1991-10-04,5,claims of policy,Whiplash,171320
N,SUBID1006,26800,1991-03-26,6,claims of fact,Sunbathing,107794
N,SUBID10007,177186,1946-09-05,7,claims of value,Drug consumption,130339
N,SUBID10008,141123,1966-06-20,8,claims of fact,Dengue,110377
N,SUBID10009,88540,1945-12-29,9,claims of value,Head banging,149367


In [0]:
df = df.drop("_id")

In [0]:
df = df.replace('NaN', None)
df = df.fillna({'claim_or_rejected': 'N'})

In [0]:
missingvaluecheck(df)

+-----------------+------+------------+----------+--------+----------+------------+----------+
|Claim_Or_Rejected|SUB_ID|claim_amount|claim_date|claim_id|claim_type|disease_name|patient_id|
+-----------------+------+------------+----------+--------+----------+------------+----------+
|                0|     0|           0|         0|       0|         0|           0|         0|
+-----------------+------+------------+----------+--------+----------+------------+----------+



In [0]:
tocheckduplicates(df)

+-----------------+------+------------+----------+--------+----------+------------+----------+-----+
|Claim_Or_Rejected|SUB_ID|claim_amount|claim_date|claim_id|claim_type|disease_name|patient_id|count|
+-----------------+------+------------+----------+--------+----------+------------+----------+-----+
+-----------------+------+------------+----------+--------+----------+------------+----------+-----+



In [0]:
df = df.withColumn('claim_amount', df['claim_amount'].cast('integer'))
df = df.withColumn('claim_date', to_date(df['claim_date'], 'yyyy-MM-dd'))

## Scbscriber Transformation

In [0]:
df = readbronzedata("subscriber.csv")

adls connection successful


In [0]:
df.display()

sub_id,first_name,last_name,Street,Birth_date,Gender,Phone,Country,City,Zip Code,Subgrp_id,Elig_ind,eff_date,term_date
SUBID10000,Harbir,Vishwakarma,Baria Marg,1924-06-30,Female,+91 0112009318,India,Rourkela,767058,S107,Y,1944-06-30,1954-01-14
SUBID10001,Brahmdev,Sonkar,Lala Marg,1948-12-20,Female,+91 1727749552,India,Tiruvottiyur,34639,S105,Y,1968-12-20,1970-05-16
SUBID10002,Ujjawal,Devi,Mammen Zila,1980-04-16,Male,+91 8547451606,India,Berhampur,914455,S106,N,2000-04-16,2008-05-04
SUBID10003,Ballari,Mishra,Sahni Zila,1969-09-25,Female,+91 0106026841,India,Bihar Sharif,91481,S104,N,1989-09-25,1995-06-05
SUBID10004,Devnath,Srivastav,Magar Zila,1946-05-01,Female,+91 1868774631,India,Bidhannagar,531742,S110,N,1966-05-01,1970-12-09
SUBID10005,Atasi,Seth,Khatri Nagar,1967-10-02,Male,+91 9747336855,India,Amravati,229062,S104,Y,1987-10-02,1995-02-13
SUBID1006,Manish,Maurya,Swaminathan Chowk,1967-06-06,Male,+91 4354294043,India,Panvel,438733,S109,null,1987-06-06,1995-03-21
SUBID10007,Aakar,Yadav,Swamy,1925-03-05,Female,+91 2777633911,India,Bihar Sharif,535907,S104,N,1945-03-05,1946-11-07
SUBID10008,Gurudas,Gupta,Sarin Nagar,1945-05-06,Male,+91 1232859381,India,Kamarhati,933226,S103,Y,1965-05-06,1970-09-16
SUBID10009,null,Gupta,Thakur Circle,1925-06-12,Male,+91 1780763280,India,Bangalore,957469,S105,Y,1945-06-12,1953-08-30


In [0]:
df_sel = df.select('sub_id', 'first_name', 'last_name', 'Street', 'Gender', 'Phone', 'Country', 'City', 'Zip Code', 'Subgrp_id', 'Elig_ind') 
missingvaluecheck(df_sel)

+------+----------+---------+------+------+-----+-------+----+--------+---------+--------+
|sub_id|first_name|last_name|Street|Gender|Phone|Country|City|Zip Code|Subgrp_id|Elig_ind|
+------+----------+---------+------+------+-----+-------+----+--------+---------+--------+
|     0|        27|        0|     0|     0|    3|      0|   0|       0|        2|       4|
+------+----------+---------+------+------+-----+-------+----+--------+---------+--------+



In [0]:
df_date = df.select('Birth_date','eff_date','term_date')
df_date.select([col(c).isNull().cast('int').alias(c) for c in df_date.columns]).agg(*[sum(col(c)).alias(c) for c in df_date.columns]).show()

+----------+--------+---------+
|Birth_date|eff_date|term_date|
+----------+--------+---------+
|         0|       0|        0|
+----------+--------+---------+



In [0]:
missingvaluecheck_alldatatypes(df)

+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+
|sub_id|first_name|last_name|Street|Birth_date|Gender|Phone|Country|City|Zip Code|Subgrp_id|Elig_ind|eff_date|term_date|
+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+
|     0|        27|        0|     0|         0|     0|    3|      0|   0|       0|        2|       4|       0|        0|
+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+



In [0]:
df = df.fillna({'first_name': 'Visitor/NA', 'Elig_ind': 'N'})
df = df.drop('Phone')
df = df.withColumn('age', round(months_between(current_date(), df.Birth_date)/12,0).cast('integer'))
df = df.drop('Birth_date')

In [0]:
missingvaluecheck_alldatatypes(df)

+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+---+
|sub_id|first_name|last_name|Street|Gender|Country|City|Zip Code|Subgrp_id|Elig_ind|eff_date|term_date|age|
+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+---+
|     0|         0|        0|     0|     0|      0|   0|       0|        2|       0|       0|        0|  0|
+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+---+



In [0]:
df.display()

sub_id,first_name,last_name,Street,Gender,Country,City,Zip Code,Subgrp_id,Elig_ind,eff_date,term_date,age
SUBID10000,Harbir,Vishwakarma,Baria Marg,Female,India,Rourkela,767058,S107,Y,1944-06-30,1954-01-14,101
SUBID10001,Brahmdev,Sonkar,Lala Marg,Female,India,Tiruvottiyur,34639,S105,Y,1968-12-20,1970-05-16,76
SUBID10002,Ujjawal,Devi,Mammen Zila,Male,India,Berhampur,914455,S106,N,2000-04-16,2008-05-04,45
SUBID10003,Ballari,Mishra,Sahni Zila,Female,India,Bihar Sharif,91481,S104,N,1989-09-25,1995-06-05,55
SUBID10004,Devnath,Srivastav,Magar Zila,Female,India,Bidhannagar,531742,S110,N,1966-05-01,1970-12-09,79
SUBID10005,Atasi,Seth,Khatri Nagar,Male,India,Amravati,229062,S104,Y,1987-10-02,1995-02-13,57
SUBID1006,Manish,Maurya,Swaminathan Chowk,Male,India,Panvel,438733,S109,N,1987-06-06,1995-03-21,58
SUBID10007,Aakar,Yadav,Swamy,Female,India,Bihar Sharif,535907,S104,N,1945-03-05,1946-11-07,100
SUBID10008,Gurudas,Gupta,Sarin Nagar,Male,India,Kamarhati,933226,S103,Y,1965-05-06,1970-09-16,80
SUBID10009,Visitor/NA,Gupta,Thakur Circle,Male,India,Bangalore,957469,S105,Y,1945-06-12,1953-08-30,100


### Lets handle the 2 null values in `Subgrp_id` column

In [0]:
df.filter(col('Subgrp_id').isNull()).show()

+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+---+
|    sub_id|first_name|last_name|   Street|Gender|Country|    City|Zip Code|Subgrp_id|Elig_ind|  eff_date| term_date|age|
+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+---+
|SUBID10022|   Prakash|      Rao|   Sachar|Female|  India|Kottayam|  180680|     NULL|       N|1943-09-15|1948-10-19|102|
|SUBID10049|   Paridhi|    Yadav|Sant Path|Female|  India|Jabalpur|  883754|     NULL|       N|1979-03-27|1985-06-01| 66|
+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+---+



In [0]:
claims_df = readbronzedata_json("claims.json")

adls connection successful


In [0]:
claims_df.filter(col('SUB_ID').isin(['SUBID10022','SUBID10049'])).show()

+-----------------+----------+--------------------+------------+----------+--------+---------------+--------------+----------+
|Claim_Or_Rejected|    SUB_ID|                 _id|claim_amount|claim_date|claim_id|     claim_type|  disease_name|patient_id|
+-----------------+----------+--------------------+------------+----------+--------+---------------+--------------+----------+
|                Y|SUBID10022|{67d5bc666ac8fd8f...|       34771|1948-05-23|      22|claims of value|           Flu|    134184|
|              NaN|SUBID10049|{67d5bc666ac8fd8f...|      159815|1983-06-20|      49| claims of fact|Bladder cancer|    121783|
+-----------------+----------+--------------------+------------+----------+--------+---------------+--------------+----------+



In [0]:
# pat_df = readbronzedata('Patient_records.csv')
# pat_df.filter(col('patient_id').isin([134184,121783])).show()

In [0]:
disease_df = readbronzedata('disease.csv')
disease_df.filter(col('disease_name').isin(['Flu','Bladder cancer'])).show()

adls connection successful
+---------+----------+--------------+
|subgrp_id|disease_id|  disease_name|
+---------+----------+--------------+
|     S107|    110039|Bladder cancer|
|     S110|    110059|           Flu|
+---------+----------+--------------+



In [0]:
df = df.withColumn('Subgrp_id', 
        when((col('Subgrp_id').isNull()) & (col('sub_id') == 'SUBID10022'), 'S110')\
        .when((col('Subgrp_id').isNull()) & (col('sub_id') == 'SUBID10049'), 'S107')\
        .otherwise(col('Subgrp_id'))
)

df.display()

sub_id,first_name,last_name,Street,Gender,Country,City,Zip Code,Subgrp_id,Elig_ind,eff_date,term_date,age
SUBID10000,Harbir,Vishwakarma,Baria Marg,Female,India,Rourkela,767058,S107,Y,1944-06-30,1954-01-14,101
SUBID10001,Brahmdev,Sonkar,Lala Marg,Female,India,Tiruvottiyur,34639,S105,Y,1968-12-20,1970-05-16,76
SUBID10002,Ujjawal,Devi,Mammen Zila,Male,India,Berhampur,914455,S106,N,2000-04-16,2008-05-04,45
SUBID10003,Ballari,Mishra,Sahni Zila,Female,India,Bihar Sharif,91481,S104,N,1989-09-25,1995-06-05,55
SUBID10004,Devnath,Srivastav,Magar Zila,Female,India,Bidhannagar,531742,S110,N,1966-05-01,1970-12-09,79
SUBID10005,Atasi,Seth,Khatri Nagar,Male,India,Amravati,229062,S104,Y,1987-10-02,1995-02-13,57
SUBID1006,Manish,Maurya,Swaminathan Chowk,Male,India,Panvel,438733,S109,N,1987-06-06,1995-03-21,58
SUBID10007,Aakar,Yadav,Swamy,Female,India,Bihar Sharif,535907,S104,N,1945-03-05,1946-11-07,100
SUBID10008,Gurudas,Gupta,Sarin Nagar,Male,India,Kamarhati,933226,S103,Y,1965-05-06,1970-09-16,80
SUBID10009,Visitor/NA,Gupta,Thakur Circle,Male,India,Bangalore,957469,S105,Y,1945-06-12,1953-08-30,100


In [0]:
missingvaluecheck_alldatatypes(df)

+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+---+
|sub_id|first_name|last_name|Street|Gender|Country|City|Zip Code|Subgrp_id|Elig_ind|eff_date|term_date|age|
+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+---+
|     0|         0|        0|     0|     0|      0|   0|       0|        0|       0|       0|        0|  0|
+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+---+

